In [1]:
import pandas as pd

df = pd.read_json("../../datas/nikto/detected.json", lines=True)

df.head(5)

,method,path,headers,uuid,peer,status,cookies,response_msg,timestamp,post_data
0,GET,/,"{'host': '127.0.0.1', 'user-agent': 'curl/7.81...",eee6754c-7a79-4143-b40a-fb4f6358f230,"{'ip': '172.18.0.1', 'port': 36070}",200,{'sess_uuid': None},"{'version': '0.6.0', 'response': {'message': {...",2025-06-14 09:02:40.155204,NaN
1,HEAD,/,"{'host': 'localhost', 'connection': 'Keep-Aliv...",eee6754c-7a79-4143-b40a-fb4f6358f230,"{'ip': '172.18.0.1', 'port': 58816}",200,{'sess_uuid': None},"{'version': '0.6.0', 'response': {'message': {...",2025-06-14 09:03:06.043339,NaN
2,GET,/,{'user-agent': 'Mozilla/5.00 (Nikto/2.1.5) (Ev...,eee6754c-7a79-4143-b40a-fb4f6358f230,"{'ip': '172.18.0.1', 'port': 58816}",200,{'sess_uuid': None},"{'version': '0.6.0', 'response': {'message': {...",2025-06-14 09:03:06.107540,NaN
3,GET,/,{'user-agent': 'Mozilla/5.00 (Nikto/2.1.5) (Ev...,eee6754c-7a79-4143-b40a-fb4f6358f230,"{'ip': '172.18.0.1', 'port': 58816}",200,{'sess_uuid': None},"{'version': '0.6.0', 'response': {'message': {...",2025-06-14 09:03:06.116424,NaN
4,GET,/bTsJgT9V,{'user-agent': 'Mozilla/5.00 (Nikto/2.1.5) (Ev...,eee6754c-7a79-4143-b40a-fb4f6358f230,"{'ip': '172.18.0.1', 'port': 58816}",200,{'sess_uuid': None},"{'version': '0.6.0', 'response': {'message': {...",2025-06-14 09:03:06.126323,NaN


In [2]:
methods_matrix = []
tmp = []
sess_uuids_methods = {
  # "{sess_uuid}": []
}
sess_uuids = []

for _, row in df.iterrows():
  sess_uuid = row.get("cookies", {}).get("sess_uuid") or row.get("response_msg", {}).get("response", {}).get("message", {}).get("sess_uuid")
  method = row["method"]

  if sess_uuid in sess_uuids:
    sess_uuids_methods[sess_uuid].append(method)
  else:
    sess_uuids_methods[sess_uuid] = [method]
    sess_uuids.append(sess_uuid)

methods_matrix = list(sess_uuids_methods.values())
# for paths in sess_uuids_methods.values():
#   methods_matrix.append(paths)

sess_uuids_methods
# methods_matrix
# print(len(methods_matrix), len(sess_uuids_methods.keys()))

{'b77670f2-1ce0-4b1d-b6e2-dc7613cfaa2c': ['GET'],
 'aacd9a91-9d29-4242-914a-751f61ab34ff': ['HEAD', 'HEAD', 'HEAD'],
 '60724bd0-7e5a-4f16-b6e8-1d4b8b8b15aa': ['GET', 'GET', 'GET'],
 'bea16c2a-a854-4850-bea2-566cb80d9dc1': ['GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GE

In [3]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth

te = TransactionEncoder()
te_ary = te.fit(methods_matrix).transform(methods_matrix)
df = pd.DataFrame(te_ary, columns=te.columns_)

df

,GET,HEAD,OPTIONS,POST,PROPFIND,PUT,SEARCH,TRACE
0,True,False,False,False,False,False,False,False
1,False,True,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
2119,True,False,False,False,False,False,False,False
2120,True,False,False,False,False,False,False,False
2121,True,False,False,False,False,False,False,False
2122,True,False,False,False,False,False,False,False


In [7]:
# frequent_itemsets = fpgrowth(df, min_support=0.3, use_colnames=True)
frequent_itemsets = fpgrowth(df, min_support=0.000001, use_colnames=True)
### alternatively:
#frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)
#frequent_itemsets = fpmax(df, min_support=0.6, use_colnames=True)

frequent_itemsets.head(20)

,support,itemsets
0,0.988230,(GET)
1,0.001412,(HEAD)
2,0.001412,(PUT)
3,0.001883,(OPTIONS)
4,0.001412,(PROPFIND)
5,0.001412,(TRACE)
6,0.003766,(POST)
7,0.000471,(SEARCH)


In [8]:
import psycopg2

conn = psycopg2.connect(database="web_honeypot_nikto", user = "postgres", password = "admin", host = "127.0.0.1", port = "5432")

print("Opened database successfully")

Opened database successfully


In [9]:
# create table
cur = conn.cursor()
cur.execute('''CREATE TABLE assoc_rules_methods (
            ID INT PRIMARY KEY     NOT NULL,
            SUPPORT           REAL    NOT NULL,
            METHOD            VARCHAR(255)     NOT NULL);''')

print("Table created successfully")

conn.commit()

Table created successfully


In [10]:
# insert data

cur = conn.cursor()

insert_query = """
    INSERT INTO assoc_rules_methods (ID, SUPPORT, METHOD)
    VALUES (%s, %s, %s)
"""

for idx, row in frequent_itemsets.iterrows():
    cur.execute(
        insert_query,
        (int(idx), float(row["support"]), str(list(row["itemsets"])))
    )

conn.commit()
print("Records created successfully")
conn.close()

Records created successfully
